<a href="https://colab.research.google.com/github/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm/blob/oscar2/notebooks/oscar_test_config_kernel_size.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wandb Install, Login, Import

In [1]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!wandb login "6f19b1e6735ebc69af24f18d5b426262416027fb"

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import wandb

# Clone Team's Code

In [4]:
!rm -r /content/csgy6953_DeepLearning_Midterm/

In [5]:
!git clone -b config_kernel_size "https://github.com/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm.git"

Cloning into 'csgy6953_DeepLearning_Midterm'...
remote: Enumerating objects: 413, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 413 (delta 82), reused 82 (delta 54), pack-reused 279
Receiving objects: 100% (413/413), 111.08 KiB | 15.87 MiB/s, done.
Resolving deltas: 100% (269/269), done.


In [6]:
!cp -r /content/csgy6953_DeepLearning_Midterm/src/ .

In [7]:
!cat src/transforms.py

import torch
import torchvision.transforms as transforms
from torchvision.transforms import autoaugment

from typing import Tuple


def make_transforms(means: torch.Tensor, std_devs: torch.Tensor) -> Tuple:
    '''
    Given a tensor of computed means and a tensor of computed standard devations,
    return's a tuple containing a train and test transform pipelines
    '''
    train_transforms = transforms.Compose([
        transforms.RandomRotation(5),
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomCrop(32, padding=2),
        transforms.ToTensor(),
        transforms.Normalize(mean=means,
                             std=std_devs)
    ])

    test_transforms = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=means,
                             std=std_devs)
    ])

    return train_transforms, test_transforms


def make_auto_transforms(means: torch.Tensor, std_devs: torch.Tensor) -> Tuple:
    '''
    Utilizes PyTorch'es Auto

# Import, Seed, Device

In [8]:
import torch
import torch.nn as nn

import time
import random

In [9]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# Load Data

In [11]:
from src.data import get_transformed_data, make_data_loaders
from src.transforms import make_auto_transforms # used to use: make_transforms

BATCH_SIZE = 512
VALID_RATIO = 0.1

train_data, valid_data, test_data = \
get_transformed_data(make_transforms=make_auto_transforms, valid_ratio=VALID_RATIO)

train_iter, valid_iter, test_iter = \
make_data_loaders(train_data, valid_data, test_data, BATCH_SIZE)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting .data/cifar-10-python.tar.gz to .data
Files already downloaded and verified
Files already downloaded and verified


# Model

In [15]:
from src.model import ResNet, StemConfig

architecture = [
    (2, 64, 0.5),
    (2, 128, 0.5),
    (2, 256, 0.5),
    (2, 512, 0.5),
]

config = StemConfig(num_channels=64, kernel_size=3, stride=1, padding=1)

model  = ResNet(architecture, stem_config=config, output_size=10)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [16]:
# intialize a new model

inputs = torch.empty((BATCH_SIZE, 3, 32, 32))
inputs.normal_()

model = model.to(device)

outputs = model(inputs.to(device)) 
# (Oscar) observation: 2022.11/13(7)_a08.14: internally, this converts all nn.LazyConv2d layers to nn.Conv2d
# to see this, run print(model) both before and after this operation

print(outputs.size())

RuntimeError: ignored

In [11]:
from src.utils import initialize_parameters, epoch_time